# Install Libraries

In [ ]:
!pip install tensorflow pandas numpy sklearn
!pip install pandas scikit-learn
!pip install lime shap

# Import Libraries

In [ ]:
import itertools
import random
import math
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN
from tensorflow.keras.metrics import AUC, FalseNegatives, FalsePositives, Precision, TrueNegatives, TruePositives
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Extract Raw Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path_AU_raw = '/content/drive/MyDrive/CKD/Finalised/data/raw/AU/eGFR 12-09-2023.xlsx'
file_path_JP_raw = '/content/drive/MyDrive/CKD/Finalised/data/raw/JP/ROUTE_proteinuria_dataset.xlsx'

# Preprocess Australian (AU) Data

## 1) Applying Inclusion Criteria

In [ ]:
df = pd.read_excel(file_path_AU_raw)
print("Total number of unique ids:", len(df['ID'].unique()))
df

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
# Replace '> 90' with 91 and convert to integer
print(set(df['eGFR']))

df['eGFR'] = np.where(df['eGFR'].str.strip() == "> 90", 91, df['eGFR']).astype(int)

print("Total number of unique ids:", len(df['ID'].unique()))
print(set(df['eGFR']))

In [ ]:
# Filter in Age 18 and over
df = df[df['AGE'] >= 18]
df.reset_index(inplace=True, drop=True)
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
# Sort by date
df.sort_values('COLLECTED', inplace=True)

In [ ]:
# Filter out patients with less than 3 eGFR measurements
df = df[df.groupby('ID').ID.transform('count') > 2]
df.reset_index(inplace=True, drop=True)
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
# Filter to only include only those patients with the first reading between 15 (patient in a dialysis) and 59
filter_1 = df.groupby('ID')['eGFR'].apply(lambda x: 15 <= x.iloc[0] <= 59) #logic is same as (first value >=15) and (first value <=59)
df = df[df['ID'].map(filter_1)] # Applying filter
df.reset_index(inplace=True, drop=True)
print("Total number of unique ids:", len(df['ID'].unique()))

grouped = df.groupby('ID')['eGFR'].first()
print(grouped)

In [ ]:
# Exclude patients who had two eGFR readings on the same day after their first eGFR reading < 15 (considered to be on dialysis)

# Group the data by 'ID' and get the date when the first eGFR < 15 for each patient
first_low_egfr_date = df[df['eGFR'] < 15].groupby('ID')['COLLECTED'].min()

# Join the dates of first low eGFR readings to the original DataFrame
df = df.join(first_low_egfr_date.rename('first_low_egfr_date'), on='ID')

# Only keep rows where 'Date' is after the 'first_low_egfr_date'
df_after_first_low_egfr = df[df['COLLECTED'] > df['first_low_egfr_date']]

# Identify patients who have 2 or more readings on any day after their first eGFR reading < 15
patients_on_dialysis = df_after_first_low_egfr.groupby(['ID', 'COLLECTED']).filter(lambda x: len(x) >= 2)['ID'].unique()

# Filter these patients out from the original DataFrame
df = df[~df['ID'].isin(patients_on_dialysis)]

df.drop('first_low_egfr_date', axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)

print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
df_copy = df.copy()

In [ ]:
#df_copy = df

## 2) Kidney Failure Criteria

In [ ]:
# Kidney failure was determined as follows: UIDs with two eGFR readings < 15 at least 30 days apart, with one eGFR reading > 15 immediately preceding these.
# This eliminates cases where patients are already in kidney failure — only patients that progress from CKD to kidney failure are included.

patient_ids = []

# Loop over unique IDs in the dataframe
for id in df['ID'].unique():

    # Extract records for this patient excluding the first reading
    patient_df = df[df['ID'] == id]

    # Find instances where eGFR is less than 15
    low_eGFR = patient_df[patient_df['eGFR'] < 15].sort_values(by='COLLECTED')

    # Check if there are at least two such instances
    if low_eGFR.shape[0] >= 2:

        # Loop through the data and find if there are any two dates at least 90 days apart
        found_90_days_apart = False
        for date1, date2 in itertools.combinations(low_eGFR['COLLECTED'], 2):
            time_diff = (date2 - date1).days

            # If time difference is at least 90 days, set the flag to True and break
            if time_diff >= 30: #90  #355 patients at 29 days
                found_90_days_apart = True
                break

        if found_90_days_apart:
            patient_ids.append(id)

# Print the total number of patients
print(f"Total number of patients: {len(patient_ids)}")

In [ ]:
# Filter the dataframe to only include patients of interest
df = df[df['ID'].isin(patient_ids)]
df.reset_index(inplace=True, drop=True)

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))
print(f"Total number of rows: {len(df)}")

In [ ]:
# # Sort the DataFrame by the ID column
# df_unique = df.drop_duplicates(subset='ID')
# sorted_df = df_unique.sort_values('ID')
# subset = sorted_df['ID']

### Further Analysis on Patient IDs with Kidney Failure

In [ ]:
# Calculate the number of records per ID
records_per_id = df['ID'].value_counts()

# Convert the Series to a DataFrame
records_per_id_df = records_per_id.reset_index()

# Rename the columns
records_per_id_df.columns = ['ID', 'Number of Records']

# Sort DataFrame by 'Number of Records' in descending order
records_per_id_df = records_per_id_df.sort_values(by='Number of Records', ascending=False)

# Display the DataFrame
print(records_per_id_df)

In [ ]:
import pandas as pd

# Calculate the number of records per ID
records_per_id = df['ID'].value_counts()

# Find the bin range that contains the most IDs
hist, bin_edges = np.histogram(records_per_id, bins=30)
max_bin_index = np.argmax(hist)
most_common_range = (bin_edges[max_bin_index], bin_edges[max_bin_index + 1])

# Output the most common range
print(f"The most common range of records per ID is: {most_common_range}")

In [ ]:
# Use fewer bins
plt.hist(records_per_id, bins=40, color="#005589", edgecolor='black')  # 'auto' lets matplotlib decide the optimal number of bins
# Or specify a fixed number of bins
# plt.hist(records_per_id, bins=10, color="#005589", edgecolor='black")  # for 10 bins

# Set the title and labels
plt.title('Distribution of Number of Records per ID')
plt.xlabel('Number of Records per ID')
plt.ylabel('Count of IDs')

# Add horizontal grid only
plt.grid(axis='y')

# Rotate x-axis labels if there are still many bins
plt.xticks(rotation=90)

# Show the plot
plt.show()

In [ ]:
# Group by 'ID' and 'COLLECTED_DATE', and count the number of records in each group
same_day_readings = df.groupby(['ID', 'COLLECTED']).size()

# Find out which IDs had multiple readings on the same day
multiple_same_day_readings = same_day_readings[same_day_readings > 1]

# Print the result
print(multiple_same_day_readings)

In [ ]:
# Group by 'ID' and find the first 'COLLECTED_DATE' for each group
first_reading_dates = df.groupby('ID')['COLLECTED'].min().reset_index()

# Rename the columns for the join operation
first_reading_dates.columns = ['ID', 'FIRST_READING_DATE']

# Join 'multiple_same_day_readings' with 'first_reading_dates' on 'ID' and 'COLLECTED_DATE'
AU_df = pd.merge(multiple_same_day_readings.reset_index(), first_reading_dates, how='left', left_on=['ID', 'COLLECTED'], right_on=['ID', 'FIRST_READING_DATE'])

# Find out how many of the multiple-reading days were the first day of readings
first_day_multiple_readings = AU_df[AU_df['COLLECTED'] == AU_df['FIRST_READING_DATE']]

# Print the result
print(first_day_multiple_readings)

## 3) Obtain Final Dataframe with the added Kidney Failure Target Variable

In [ ]:
df = df_copy

In [ ]:
# Create a new column 'Kidney Failure'. If the patient ID is in the list, assign 1, else 0
df['Kidney_Failure'] = df['ID'].isin(patient_ids).astype(int)
df.reset_index(inplace=True, drop=True)
df

In [ ]:
df.sort_values(by=['ID', 'COLLECTED'], inplace=True)
df.reset_index(inplace=True, drop=True)
df

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))
print(f"Total number of rows: {len(df)}")
print("Total number of unique ids with KF:", len(df[df['Kidney_Failure']==1]['ID'].unique()))
print(f"Total number of rows with KF: {len(df[df['Kidney_Failure']==1])}")

In [ ]:
# Truncate onwards for each ID where the eGFR first drops below 15.
mask = df.groupby('ID')['eGFR'].transform(lambda x: x.ge(15).cummin())
df = df.loc[mask]
df.reset_index(inplace=True, drop=True)
df

In [ ]:
AU_df = df

# Final checks

# Convert 'COLLECTED' to datetime and sort the dataframe by 'ID' and 'COLLECTED'
AU_df['COLLECTED'] = pd.to_datetime(AU_df['COLLECTED'])
AU_df.sort_values(['ID', 'COLLECTED'], inplace=True)

# Encode 'SEX' as 2 for female and 1 for male to match JP dataset
AU_df['SEX'] = AU_df['SEX'].replace({'F': 2, 'M': 1})

# Drop extraneous columns
AU_df = AU_df.drop(columns=['INPATIENT/OUTPATIENT'])

In [ ]:
print("Total number of unique ids:", len(AU_df['ID'].unique()))
print(f"Total number of rows: {len(AU_df)}")
print("Total number of unique ids with KF:", len(AU_df[AU_df['Kidney_Failure']==1]['ID'].unique()))
print(f"Total number of rows with KF: {len(AU_df[AU_df['Kidney_Failure']==1])}")

# Find unique male and female IDs
unique_male_ids = AU_df[AU_df['SEX'] == 1]['ID'].unique()
unique_female_ids = AU_df[AU_df['SEX'] == 2]['ID'].unique()

# Count these unique IDs
male_cases = len(unique_male_ids)
female_cases = len(unique_female_ids)

# Print the counts
print(f"Number of unique male cases: {male_cases}")
print(f"Number of unique female cases: {female_cases}")

# Filter out the rows where Kidney_Failure is 1
kidney_failure_df = AU_df[AU_df['Kidney_Failure'] == 1]

# Find unique male and female IDs with kidney failure
unique_male_ids = kidney_failure_df[kidney_failure_df['SEX'] == 1]['ID'].unique()
unique_female_ids = kidney_failure_df[kidney_failure_df['SEX'] == 2]['ID'].unique()

# Count these unique IDs
male_cases = len(unique_male_ids)
female_cases = len(unique_female_ids)

# Print the counts
print(f"Number of unique male cases of kidney failure: {male_cases}")
print(f"Number of unique female cases of kidney failure: {female_cases}")

In [ ]:
AU_df.to_csv('/content/drive/MyDrive/CKD/Finalised/data/preprocessed/AU/AU_preprocessed_df.csv', index=False)

# Preprocess Japanese (JP) Data

## 1) Additional Pre-steps

In [ ]:
df = pd.read_excel(file_path_JP_raw)
df

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))
print(f"Total number of rows: {len(df)}")

In [ ]:
columns_to_flatten = ['eGFR(0M)', 'eGFR(6M)', 'eGFR(12M)', 'eGFR(18M)', 'eGFR(24M)', 'eGFR(30M)', 'eGFR(36M)', 'eGFR(last visit)']

# Use melt to flatten these columns
df_melted = pd.melt(df, id_vars=['ID', 'age', 'gender', 'Cr'], value_vars=columns_to_flatten, var_name='Time', value_name='eGFR')

# Display the melted DataFrame
df_melted

In [ ]:
df_melted.isna().sum()

In [ ]:
import math

# Update age based on time of collection
def amend_age(row):
    time_str = row['Time']
    months = 0
    if time_str == 'eGFR(6M)':
        months = 6
    elif time_str == 'eGFR(12M)':
        months = 12
    elif time_str == 'eGFR(18M)':
        months = 18
    elif time_str == 'eGFR(24M)':
        months = 24
    elif time_str == 'eGFR(30M)':
        months = 30
    elif time_str == 'eGFR(36M)':
        months = 36
    elif time_str == 'eGFR(last visit)':
        months = 42  # Adding 42 months for 'last visit'

    amended_age = row['age'] + (months / 12.0)

    # Round down the amended age
    return math.floor(amended_age)

# Apply the function to the DataFrame
df_melted['amended_age'] = df_melted.apply(amend_age, axis=1)
df_melted

In [ ]:
# Calculate SCr
def calculate_SCr(row):
    age_factor = row['amended_age'] ** -0.287
    gender_factor = 0.739 if row['gender'] == 2 else 1  # assuming 'female' and 'male' are the gender labels
    eGFR = row['eGFR']

    SCr = ((eGFR / (194 * age_factor * gender_factor)) ** (-1 / 1.094))
    return SCr

# Apply the function to create a new column 'Calculated_SCr'
df_melted['Calculated_SCr'] = df_melted.apply(calculate_SCr, axis=1)

# Display the DataFrame to check the results
df_melted

In [ ]:
# Convert to Cr to umol/L

df_melted['Cr_umol_per_L'] = df_melted['Calculated_SCr'] * 88.4
df_melted

In [ ]:
def calculate_eGFR_CKD_EPI(row):
    if row['gender'] == 2:
        if row['Cr_umol_per_L'] <= 62:
            return 144 * ((row['Cr_umol_per_L'] * 0.0113 / 0.7) ** -0.329) * ((0.993) ** row['amended_age'])
        else:
            return 144 * ((row['Cr_umol_per_L'] * 0.0113 / 0.7) ** -1.209) * ((0.993) ** row['amended_age'])
    else:  # Assuming gender is male if not female
        if row['Cr_umol_per_L'] <= 80:
            return 141 * ((row['Cr_umol_per_L'] * 0.0113 / 0.9) ** -0.411) * ((0.993) ** row['amended_age'])
        else:
            return 141 * ((row['Cr_umol_per_L'] * 0.0113 / 0.9) ** -1.209) * ((0.993) ** row['amended_age'])

# Apply the function to create a new column 'eGFR_CKD_EPI'
df_melted['eGFR_CKD_EPI'] = df_melted.apply(calculate_eGFR_CKD_EPI, axis=1)

# Display the DataFrame to check the results
df_melted

In [ ]:
df_melted.isna().sum()

In [ ]:
# Filter the DataFrame to only include rows where 'eGFR_CKD_EPI' is NaN
na_eGFR_CKD_EPI_df = df_melted[df_melted['eGFR_CKD_EPI'].isna()]
na_eGFR_CKD_EPI_df.reset_index(inplace=True, drop=True)
na_eGFR_CKD_EPI_df

In [ ]:
# Drop NaN subset
df_melted.dropna(subset=['eGFR_CKD_EPI'], inplace=True)
df_melted.reset_index(inplace=True, drop=True)
df

In [ ]:
# Only Checking Kidney Failure Instances

# Filter the DataFrame to include only rows where eGFR_CKD_EPI is less than 15
filtered_df = df_melted[df_melted['eGFR_CKD_EPI'] < 15]

# Group by gender and count the number of cases
grouped_df = filtered_df.groupby('gender').size()

# Display the count of male and female cases
print("Number of male cases where eGFR_CKD_EPI < 15:", grouped_df.get(1, 0))
print("Number of female cases where eGFR_CKD_EPI < 15:", grouped_df.get(2, 0))

In [ ]:
# Drop the specified columns
df_melted.drop(['age', 'Cr', 'eGFR', 'Calculated_SCr', 'Cr_umol_per_L'], axis=1, inplace=True)
df_melted.reset_index(inplace=True, drop=True)
df_melted

In [ ]:
# Rename columns
df_melted.rename(columns={
    'amended_age': 'AGE',
    'gender': 'SEX',
    'eGFR_CKD_EPI': 'eGFR'
}, inplace=True)

df_melted

## 2) Applying Inclusion Criteria (similar to AU)

In [ ]:
df = df_melted.copy()
df

In [ ]:
df.dtypes

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
# Filter in Age 18 and over
df = df[df['AGE'] >= 18]
df.reset_index(inplace=True, drop=True)
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
# Sort by date
df.sort_values('ID', inplace=True)

In [ ]:
# Filter out patients with less than 3 eGFR measurements
df = df[df.groupby('ID').ID.transform('count') > 2]
df.reset_index(inplace=True, drop=True)
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
1138 - 939

In [ ]:
# Categorical sort by time of collection
time_order = ['eGFR(0M)', 'eGFR(6M)', 'eGFR(12M)', 'eGFR(18M)', 'eGFR(24M)', 'eGFR(30M)', 'eGFR(36M)', 'eGFR(last visit)']
df['Time'] = pd.Categorical(df['Time'], categories=time_order, ordered=True)
df = df.sort_values(['ID', 'Time'])
df.reset_index(inplace=True, drop=True)
df

In [ ]:
# Filter to only include first readings between 15 (patient in a dialysis) and 59
filter_1 = df.groupby('ID')['eGFR'].apply(lambda x: 15 <= x.iloc[0] <= 59) #logic is same as (first value >=15) and (first value <=59)
df = df[df['ID'].map(filter_1)] # Applying filter
df.reset_index(inplace=True, drop=True)

print("Total number of unique ids:", len(df['ID'].unique()))

grouped = df.groupby('ID')['eGFR'].first()
print(grouped)

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))

In [ ]:
df_copy = df.copy()

In [ ]:
# Filter the DataFrame to include only rows where eGFR_CKD_EPI is less than 15
filtered_df = df[df['eGFR'] < 15]

# Group by gender and count the number of cases
grouped_df = filtered_df.groupby('SEX').size()

# Display the count of male and female cases
print("Number of male cases where eGFR_CKD_EPI < 15:", grouped_df.get(1, 0))
print("Number of female cases where eGFR_CKD_EPI < 15:", grouped_df.get(2, 0))

## 3) Kidney Failure Criteria (similar to AU)

In [ ]:
patient_ids = []

# Loop over unique IDs in the dataframe
for id in df['ID'].unique():

    # Extract records for this patient excluding the first reading (eGFR(0M))
    patient_df = df[(df['ID'] == id) & (df['Time'] != 'eGFR(0M)')]

    # Find instances where eGFR is less than 15
    low_eGFR = patient_df[patient_df['eGFR'] < 15]

    # Check if there are at least two such instances
    if low_eGFR.shape[0] >= 2:
        patient_ids.append(id)

# Print the total number of patients
print(f"Total number of patients: {len(patient_ids)}")

In [ ]:
# Filter the dataframe to only include patients of interest
df = df[df['ID'].isin(patient_ids)]

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))
print(f"Total number of rows: {len(df)}")

In [ ]:
# Filter the DataFrame to include only rows where eGFR_CKD_EPI is less than 15
filtered_df = df[df['eGFR'] < 15]

# Group by gender and count the number of cases
grouped_df = filtered_df.groupby('SEX').size()

# Display the count of male and female cases
print("Number of male cases where eGFR_CKD_EPI < 15:", grouped_df.get(1, 0))
print("Number of female cases where eGFR_CKD_EPI < 15:", grouped_df.get(2, 0))

### Further Analysis on Patient IDs with Kidney Failure

In [ ]:
# Calculate the number of records per ID
records_per_id = df['ID'].value_counts()

# Convert the Series to a DataFrame
records_per_id_df = records_per_id.reset_index()

# Rename the columns
records_per_id_df.columns = ['ID', 'Number of Records']

# Sort DataFrame by 'Number of Records' in descending order
records_per_id_df = records_per_id_df.sort_values(by='Number of Records', ascending=False)

# Display the DataFrame
print(records_per_id_df)

In [ ]:
# Use fewer bins
plt.hist(records_per_id, bins='auto', color="#005589", edgecolor='black')  # 'auto' lets matplotlib decide the optimal number of bins
# Or specify a fixed number of bins
# plt.hist(records_per_id, bins=10, color="#005589", edgecolor='black")  # for 10 bins

# Set the title and labels
plt.title('Distribution of Number of Records per ID')
plt.xlabel('Number of Records per ID')
plt.ylabel('Count of IDs')

# Add horizontal grid only
plt.grid(axis='y')

# Rotate x-axis labels if there are still many bins
plt.xticks(rotation=90)

# Show the plot
plt.show()


## 4) Obtain Final Dataframe with the added Kidney Failure Target Variable (similar to AU)

In [ ]:
df = df_copy

In [ ]:
# Create a new column 'Kidney Failure'. If the patient ID is in the list, assign 1, else 0
df['Kidney_Failure'] = df['ID'].isin(patient_ids).astype(int)
df.reset_index(inplace=True, drop=True)
df

In [ ]:
print("Total number of unique ids:", len(df['ID'].unique()))
print(f"Total number of rows: {len(df)}")
print("Total number of unique ids with KF:", len(df[df['Kidney_Failure']==1]['ID'].unique()))
print(f"Total number of rows with KF: {len(df[df['Kidney_Failure']==1])}")

In [ ]:
# Filter out the rows where Kidney_Failure is 1
kidney_failure_df = df[df['Kidney_Failure'] == 1]

# Find unique male and female IDs with kidney failure
unique_male_ids = kidney_failure_df[kidney_failure_df['SEX'] == 1]['ID'].unique()
unique_female_ids = kidney_failure_df[kidney_failure_df['SEX'] == 2]['ID'].unique()

# Count these unique IDs
male_cases = len(unique_male_ids)
female_cases = len(unique_female_ids)

# Print the counts
print(f"Number of unique male cases of kidney failure: {male_cases}")
print(f"Number of unique female cases of kidney failure: {female_cases}")

In [ ]:
#df.sort_values(by=['ID', 'COLLECTED'], inplace=True)
df.isna().sum()

In [ ]:
# Truncate onwards for each ID where the eGFR first drops below 15.
mask = df.groupby('ID')['eGFR'].transform(lambda x: x.ge(15).cummin())
df = df.loc[mask]
df.reset_index(inplace=True, drop=True)
df

In [ ]:
JP_df = df.copy()
JP_df

In [ ]:
# STATS
print("Total number of unique ids:", len(JP_df['ID'].unique()))
print(f"Total number of rows: {len(JP_df)}")
print("Total number of unique ids with KF:", len(JP_df[JP_df['Kidney_Failure']==1]['ID'].unique()))
print(f"Total number of rows with KF: {len(JP_df[JP_df['Kidney_Failure']==1])}")

# Find unique male and female IDs with kidney failure
unique_male_ids = JP_df[JP_df['SEX'] == 1]['ID'].unique()
unique_female_ids = JP_df[JP_df['SEX'] == 2]['ID'].unique()

# Count these unique IDs
male_cases = len(unique_male_ids)
female_cases = len(unique_female_ids)

# Print the counts
print(f"Number of unique male cases of kidney failure: {male_cases}")
print(f"Number of unique female cases of kidney failure: {female_cases}")

# Filter out the rows where Kidney_Failure is 1
kidney_failure_df = JP_df[JP_df['Kidney_Failure'] == 1]

# Find unique male and female IDs with kidney failure
unique_male_ids = kidney_failure_df[kidney_failure_df['SEX'] == 1]['ID'].unique()
unique_female_ids = kidney_failure_df[kidney_failure_df['SEX'] == 2]['ID'].unique()

# Count these unique IDs
male_cases = len(unique_male_ids)
female_cases = len(unique_female_ids)

# Print the counts
print(f"Number of unique male cases of kidney failure: {male_cases}")
print(f"Number of unique female cases of kidney failure: {female_cases}")

In [ ]:
# Export data
JP_df.to_csv('/content/drive/MyDrive/CKD/Finalised/data/preprocessed/JP/JP_preprocessed_df.csv', index=False)

# END